<a href="https://colab.research.google.com/github/youkiti/ARE/blob/main/Terao_Data_extraction_from_fulltext_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is a code to simplify the full-text review of a PDF file using OpenAI's GPT-4 API.

##Inspiration: https://onlinelibrary.wiley.com/doi/10.1002/jrsm.1689?af=R
##Requirements:

*   pdf files in Google drive
*   OpenAI API key





In [ ]:
!pip install -q openai pypdf2 llmx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#configuration

In [ ]:
from openai import OpenAI
from openai import AzureOpenAI
import json
import pandas as pd
import os
import glob
import numpy as np
import PyPDF2
from tqdm import tqdm
from google.colab import userdata

#If you use Azure
#key
API_KEY = userdata.get('Azure_API_key') #2024-02-15-preview
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')
AZURE_API_version = userdata.get('AZURE_API_version')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=AZURE_API_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)



#deploy names
deployment_ids = ["gpt-4-turbo","gpt-35-turbo","text-embedding-ada-002"]

#data preparation

In [ ]:
folder_path = "/content/drive/MyDrive/2024年/01寺尾先生/包含研究"
os.chdir(folder_path)
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))

In [ ]:
print(f"the number of pdf files:{len(pdf_files)}")

the number of pdf files:42


In [ ]:
#extract text
def pdf_extract(pdf_file):
  data = []
  with open(pdf_file, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      # concatenate
      text = ''
      for page in reader.pages:
        text += page.extract_text()
  return text

#Process of GPT

In [ ]:
#To modify the function to extract specific information from a PDF, you should adjust it according to your needs. Ensure that the modifications align with the function calling and settings you plan to use.

message1 = """
You are a systematic reviewer. Please extract the key features from the user inputted research article.
Specifically,the user needs information on the first author, publication year, diagnostic criteria, depression measurement scale, response and remission criteria,  concomitant psychotropic medication (Yes/No), sources of funding, study’s setting, timeframe for follow-up and interventions.
In addition, the following data are required to be extracted separately for each intervention Number of failed antidepressant trials, interventions, sample size, age (mean), female (%), relapse endpoints, baseline depression severity (raw score), hospitalization (%), proportion of patients with psychotic symptoms (%), proportion of patients with bipolar disorder (%), proportion of patients with a history of intravenous racemic ketamine (%), proportion of patients with a history of electroconvulsive therapy (%), proportion of patients with a history of repetitive transcranial magnetic stimulation (%).
"""




In [ ]:
message2 = """
You are a systematic reviewer. Please extract the key features from the user inputted research article.
The user will need the following outcome data, separately for each intervention: (1) Response rates, as defined by the individual studies. For outcome measurement scales, we will use the following predefined hierarchy: The Montgomery-Åsberg Depression Rating Scale was used first, then the Hamilton Rating Scale for Depression, and finally the Beck Depression Inventory. (2) Remission rates, as defined by individual studies. For outcome scales, we will use the following predefined hierarchy The Montgomery-Åsberg Depression Rating Scale was used first, followed by the Hamilton Rating Scale for Depression, and finally the Beck Depression Inventory. (3) Tolerability, defined as dropout rates due to adverse effects. (4) Acceptability, defined as dropout rates for any reason. (5) Change in cognitive function between baseline and endpoints. We will select outcome measures according to a predetermined hierarchy in the following order: Mini Mental State Examination (MMSE), Alzheimer's Disease Assessment Scale - Cognitive subscale (ADAS-Cog), and Montreal Cognitive Assessment (MoCA). If none of these were used, a scale measuring global cognitive function was used. (6) Relapse rates, as defined by each study.
"""


In [ ]:
#Ask GPT to write the JSON of function calling

system_for_function = """
Write a JSON structure for a function specification that includes the name of the function, a description, and detailed parameters based on the requirements listed in the system message. Each parameter should have a type and a brief description. Format the JSON as you would for API documentation.

#example of a system message
system_message = You are a systematic reviewer. Please extract the key features from the user inputted research article. User need information about the title of the article, publication year.

#example of a function specification
function=[
    {
       "type": "function",
       "function":{
        "name":"data_extraction",
        "description":"characteristics of the study",
        "parameters":{
            "type":"object",
            "properties":{
                "title":{
                    "type":"string",
                    "description":"the title of the research article"
                },
                "publication year":{
                    "type":"string",
                    "description":"the publication year of the research article"
                }
            }
        }
    }
    }
]

"""

"""
function = [
    {
        "type": "function",
       "function":{
        "name":"json_output",
        "description":" a JSON structure for a function specification",
        "parameters":{
            "type":"object",
            "properties":{
                "json":{
                    "type":"string",
                    "description":"JSON"
                }
            }
        }
    }
    }
]
"""


res = client.chat.completions.create(
    model=deployment_ids[0],
    messages=[
        {"role":"system",
        "content":system_for_function},
        {
        "role": "user",
        "content": message1
        }],
    #tools = functions,
    #tool_choice={"type": "function", "function": {"name": "json_output"}},
    response_format = { "type": "json_object" },
    temperature = 0,
    max_tokens = 4000
  )


In [ ]:
res2 = client.chat.completions.create(
    model=deployment_ids[0],
    messages=[
        {"role":"system",
        "content":system_for_function},
        {
        "role": "user",
        "content": message2
        }],
    #tools = functions,
    #tool_choice={"type": "function", "function": {"name": "json_output"}},
    response_format = { "type": "json_object" },
    temperature = 0,
    max_tokens = 4000
  )

In [ ]:
json_string = res.choices[0].message.content
json_string2 = res2.choices[0].message.content

In [ ]:
# JSON文字列を辞書に変換
data1 = json.loads(json_string)
data2 = json.loads(json_string2)

#data1の情報
function_name = data1["function"]["name"]
function_description = data1["function"]["description"]

In [ ]:
data1_parameters = data1['function']["parameters"]
data2_parameters = data2["function"]["parameters"]

# combined_parametersを初期化
combined_parameters = {}

# data1のパラメータを追加
for key, value in data1_parameters.items():
    if key in combined_parameters:
        # キーが既に存在する場合は、値をリストに追加
        if not isinstance(combined_parameters[key], list):
            combined_parameters[key] = [combined_parameters[key]]
        combined_parameters[key].append(value)
    else:
        combined_parameters[key] = value

# data2のパラメータを追加
for key, value in data2_parameters.items():
    if key in combined_parameters:
        # キーが既に存在する場合は、値をリストに追加
        if not isinstance(combined_parameters[key], list):
            combined_parameters[key] = [combined_parameters[key]]
        combined_parameters[key].append(value)
    else:
        combined_parameters[key] = value

In [ ]:


# 結合したparametersを持つ新しい辞書を作成
combined_json = {
    "type": "function",
    "function": {
        "name": function_name,
        "description": function_description,
        "parameters": combined_parameters
    }
}

In [ ]:

#functionを設定
tools = [combined_json]

In [ ]:
tools

[{'type': 'function',
  'function': {'name': 'extract_study_features',
   'description': 'Extracts key features from a user-inputted research article for systematic review purposes.',
   'parameters': {'type': ['object', 'object'],
    'properties': [{'first_author': {'type': 'string',
       'description': 'The first author of the research article.'},
      'publication_year': {'type': 'string',
       'description': 'The publication year of the research article.'},
      'diagnostic_criteria': {'type': 'string',
       'description': 'The diagnostic criteria used in the study.'},
      'depression_measurement_scale': {'type': 'string',
       'description': 'The scale used to measure depression in the study.'},
      'response_remission_criteria': {'type': 'string',
       'description': 'The criteria used to define response and remission in the study.'},
      'concomitant_psychotropic_medication': {'type': 'string',
       'description': 'Indicates if concomitant psychotropic medic

↑Even if you use function calling, the output will not be stable, so please visually check that the format is correct.

In [ ]:
def GPT_extract(text,function_name):
    res = client.chat.completions.create(
        model=deployment_ids[0],
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": text}
        ],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": function_name}},
        temperature=0
    )



"""
    response_json = res.model_dump_json()
    try:
        # response_jsonをデコードしてからfunction_callを取得
        response_json_decoded = json.loads(response_json)
        function_call = response_json_decoded["choices"][0]["message"]["function_call"]
        if function_call is None:  # function_callがNoneであればcontentを返す
            content = response_json_decoded["choices"][0]["message"]['content']
            return content
        return function_call
    except (json.JSONDecodeError, KeyError, TypeError):
        # 例外が発生した場合はエラーメッセージを返す
        return "Error: Unable to retrieve function_call or content from response."
"""


'\n    response_json = res.model_dump_json()\n    try:\n        # response_jsonをデコードしてからfunction_callを取得\n        response_json_decoded = json.loads(response_json)\n        function_call = response_json_decoded["choices"][0]["message"]["function_call"]\n        if function_call is None:  # function_callがNoneであればcontentを返す\n            content = response_json_decoded["choices"][0]["message"][\'content\']\n            return content\n        return function_call\n    except (json.JSONDecodeError, KeyError, TypeError):\n        # 例外が発生した場合はエラーメッセージを返す\n        return "Error: Unable to retrieve function_call or content from response."\n'

In [ ]:
text = pdf_extract(pdf_files[0])

In [ ]:
res = GPT_extract(text,function_name)

In [ ]:
df = pd.DataFrame()
for pdf_file in tqdm(pdf_files[:5]):
  text = pdf_extract(pdf_file)
  response_json = GPT_extract(text,function_name)
  if response_json is None or not isinstance(response_json, dict):
      # response_jsonがNoneまたは辞書型でない場合、エラーをデータフレームに追加
      df = pd.concat([df, pd.DataFrame([{"error": response_json}])], ignore_index=True)
      continue

  arguments_str = response_json.get('arguments')
  if arguments_str is None:
      # argumentsキーが存在しない場合、エラーをデータフレームに追加
      df = pd.concat([df, pd.DataFrame([{"error": response_json}])], ignore_index=True)
      continue

  arguments_json = json.loads(arguments_str)
  df_pdf = pd.DataFrame([arguments_json])
  df = pd.concat([df, df_pdf], ignore_index=True)

In [ ]:
#Add the pdf file names
pdf_filenames = [os.path.basename(pdf_path) for pdf_path in pdf_files]
df["pdf_name"] = pdf_filenames[:5]

In [ ]:
df.head()

,first_author,publication_year,diagnostic_criteria,depression_measurement_scale,response_remission_criteria,concomitant_psychotropic_medication,sources_of_funding,study_setting,timeframe_for_follow_up,interventions,error,pdf_name
0,Robert M. Berman,2000,DSM-IV criteria for major depressive episode,25-item Hamilton Depression Rating Scale (HDRS),"Categorical 'response' criteria (i.e., 50% dec...",False,"VA Merit Award, NIMH Program Grant, and the St...","Yale University School of Medicine, Department...",2 weeks,"[{'intervention_name': 'active rTMS', 'number_...",NaN,Berman2000.pdf
1,David H. Avery,2006,DSM-IV criteria for current major depressive d...,17-item Hamilton Depression Rating Scale (HDRS),Response: ≥50% decrease in HDRS score at both ...,True,"National Institute of Mental Health, RO1 MH 62...","University of Washington School of Medicine, H...",6 months,[{'intervention_name': 'Active repetitive TMS'...,NaN,Avery2006.pdf
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,**Key Features from the Research Article:**\n\...,Blumberger2016（２０１５と同一）.pdf
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,### Key Features Extraction:\n\n- **First Auth...,Bakim2012（２０１６と同一）.pdf
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Given the extensive details provided in the us...,Ahmed2023.pdf


In [ ]:
#save
df.to_csv('data_extraction.csv',index=False, encoding='utf-8')